# Image Segmentation Using the Berkeley Segmentation Dataset (BSDS)

<img src= "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQorh8yqNAjHPP7JurjNB3xgn5-aVmypC4t-A&s" width=600>

## 🎯 Project Objective

This project aims to design, implement, and evaluate deep learning–based image segmentation models using the **Berkeley Segmentation Dataset (BSDS500)**.

The main objectives of the project are:

- To build a **Baseline Convolutional Neural Network (CNN)** for pixel-level boundary segmentation without using transfer learning.  
- To develop an **Advanced Segmentation Model using Transfer Learning with ResNet50**, including fine-tuning.  
- To preprocess image–mask pairs and convert raw human annotations into usable binary segmentation masks.  
- To compare the performance of the baseline CNN and the transfer learning model.  
- To deploy trained models using **Streamlit applications** for interactive inference.

This project focuses on understanding the **end-to-end image segmentation pipeline**, from data preprocessing to model evaluation and deployment.


In [ ]:
print("session started")

In [ ]:
import os

import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
from PIL import Image

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dropout

In [ ]:
print(tf.__version__)

In [ ]:
img_path = "/kaggle/input/berkeley-segmentation-dataset-500-bsds500/images/"
gt_path  = "/kaggle/input/berkeley-segmentation-dataset-500-bsds500/ground_truth/"

In [ ]:
os.listdir(img_path)

In [ ]:
os.listdir(gt_path)

In [ ]:
labels = ["train", "val", "test"]

In [ ]:
img_list = []
mask_list = []
label_list = []

for label in labels:
    img_dir = os.path.join(img_path, label)
    gt_dir = os.path.join(gt_path, label)

    for img_file in os.listdir(img_dir):
        if img_file.lower().endswith((".jpg", ".png", ".jpeg")):
            img_full_path = os.path.join(img_dir, img_file)
            name, _ = os.path.splitext(img_file)
            mask_file = name + ".mat"
            mask_full_path = os.path.join(gt_dir, mask_file)

            img_list.append(img_full_path)
            mask_list.append(mask_full_path)
            label_list.append(label)

## 📁 Dataset Description

The **Berkeley Segmentation Dataset 500 (BSDS500)** is a widely used benchmark dataset for image segmentation and boundary detection tasks.

Dataset characteristics:

- Total number of images: **500**  
- Training set: **200 images**  
- Validation set: **100 images**  
- Test set: **200 images**

Each image is annotated by multiple human annotators.  
Ground-truth annotations are provided in `.mat` files containing boundary maps for each annotator.

For this project, multiple boundary annotations per image are combined into a **single binary boundary mask**.

In [ ]:
df = pd.DataFrame({"img": img_list,"mask": mask_list,"label": label_list})

In [ ]:
df.shape

In [ ]:
df["label"].value_counts()

In [ ]:
plt.imshow(cv2.cvtColor(cv2.imread(df.iloc[0]["img"]), cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
d_map = {"train": 0,"val": 1,"test": 2}
df["encode_label"] = df["label"].map(d_map)

In [ ]:
df.isnull().sum()

In [ ]:
images = []

for img_path in img_list[:3]:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)

In [ ]:
plt.figure(figsize=(8, 3))

for i, image in enumerate(images):
    plt.subplot(1, 3, i + 1)
    plt.imshow(image)
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
IMG_SIZE = (168, 168)

x = []
for img_path in df["img"]:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = img.astype(np.float32) / 255.0
    x.append(img)

In [ ]:
x = np.array(x, dtype=np.float32)

In [ ]:
y = []

for mask_path in df["mask"]:
    mat = sio.loadmat(mask_path)
    gts = mat["groundTruth"][0]
    edges = []
    for g in gts:
        edges.append(g[0][0][1])  
    mask = np.max(edges, axis=0)
    mask = cv2.resize(mask, (168, 168), interpolation=cv2.INTER_NEAREST)
    mask = (mask > 0).astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    y.append(mask)

In [ ]:
y = np.array(y, dtype=np.float32)

In [ ]:
df_train = df[df["label"] == "train"].reset_index(drop=True)
df_val   = df[df["label"] == "val"].reset_index(drop=True)
df_test  = df[df["label"] == "test"].reset_index(drop=True)

In [ ]:
train_idx = df.index[df["label"] == "train"].to_numpy()
val_idx   = df.index[df["label"] == "val"].to_numpy()
test_idx  = df.index[df["label"] == "test"].to_numpy()

In [ ]:
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val     = x[val_idx], y[val_idx]
x_test, y_test   = x[test_idx], y[test_idx]

## 🧠 Baseline Model: Custom CNN

A custom **Encoder–Decoder Convolutional Neural Network (CNN)** is implemented as the baseline segmentation model.

Key characteristics of the baseline CNN:

- Sequential encoder–decoder architecture.  
- Convolutional layers followed by Batch Normalization and Max Pooling for feature extraction.  
- Upsampling layers for spatial resolution recovery.  
- Sigmoid activation function for binary segmentation output.  
- Binary Cross-Entropy loss function.  
- Regularization using **Early Stopping** and **ReduceLROnPlateau** callbacks.

This model is trained **from scratch**, without any pretrained weights, and serves as a baseline for comparison.

In [ ]:
model = Sequential()

model.add(Input(shape=(168, 168, 3)))
model.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.1))
model.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(256, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(512, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Conv2D(1, (1, 1), activation="sigmoid", padding="same"))

In [ ]:
def dice_loss(y_true, y_pred, smooth=1.0):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    y_true_f = tf.reshape(y_true, [tf.shape(y_true)[0], -1])
    y_pred_f = tf.reshape(y_pred, [tf.shape(y_pred)[0], -1])

    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=1)
    union = tf.reduce_sum(y_true_f, axis=1) + tf.reduce_sum(y_pred_f, axis=1)

    dice = (2.0 * intersection + smooth) / (union + smooth)
    return 1.0 - dice  

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    bce = tf.reduce_mean(bce, axis=[1, 2])                     
    d = dice_loss(y_true, y_pred)                              
    return tf.reduce_mean(bce + d) 

In [ ]:
model.compile(
    optimizer="adam",
    loss=bce_dice_loss)

In [ ]:
es = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True)

rlr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    min_lr=1e-6)

In [ ]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=30,
    batch_size=8,
    callbacks=[es, rlr])

In [ ]:
pred = model.predict(x_val[:5])

plt.figure(figsize=(12, 6))
for i in range(5):
    plt.subplot(5, 3, 3*i+1)
    plt.imshow(x_val[i])
    plt.title("Image")
    plt.axis("off")

    plt.subplot(5, 3, 3*i+2)
    plt.imshow(y_val[i].squeeze(), cmap="gray")
    plt.title("GT")
    plt.axis("off")

    plt.subplot(5, 3, 3*i+3)
    plt.imshow((pred[i].squeeze() > 0.10), cmap="gray")
    plt.title("Pred (t=0.10)")
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
model.summary()

In [ ]:
model.evaluate(x_test, y_test, batch_size=8)

In [ ]:
model.save("cnn_model.keras")

## 🚀 Transfer Learning with ResNet50

For the advanced model, **ResNet50 pretrained on ImageNet** is used as the encoder backbone.

Transfer learning strategy:

- A pretrained ResNet50 network is used with `include_top=False`.  
- The backbone is initially **frozen** to preserve pretrained feature representations.  
- A custom CNN-based decoder with upsampling layers is added on top of the encoder.  
- After initial training, **fine-tuning** is performed by unfreezing the last layers of ResNet50.  
- Binary Cross-Entropy loss is used for segmentation training.

This approach leverages pretrained semantic features to improve segmentation performance compared to the baseline CNN.

In [ ]:
IMG_SIZE = (224, 224)

x224 = []
for img_path in df["img"]:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = img.astype(np.float32)
    img = preprocess_input(img)         
    x224.append(img)

x224 = np.array(x224, dtype=np.float32)


y224 = []
for mask_path in df["mask"]:
    mat = sio.loadmat(mask_path)
    gts = mat["groundTruth"][0]

    edges = []
    for g in gts:
        edges.append(g[0][0][1])

    mask = np.max(edges, axis=0)

    mask = cv2.resize(mask, IMG_SIZE, interpolation=cv2.INTER_NEAREST)
    mask = (mask > 0).astype(np.float32)                              
    mask = np.expand_dims(mask, axis=-1)

    y224.append(mask)

y224 = np.array(y224, dtype=np.float32)


x224_train, y224_train = x224[train_idx], y224[train_idx]
x224_val,   y224_val   = x224[val_idx],   y224[val_idx]
x224_test,  y224_test  = x224[test_idx],  y224[test_idx]

print("Train:", x224_train.shape, y224_train.shape)
print("Val  :", x224_val.shape, y224_val.shape)
print("Test :", x224_test.shape, y224_test.shape)

In [ ]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3))

inputs = layers.Input(shape=(224, 224, 3))
z = base_model(inputs, training=False)  

z = layers.UpSampling2D((2, 2))(z)    
z = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(z)
z = layers.BatchNormalization()(z)

z = layers.UpSampling2D((2, 2))(z)     
z = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(z)
z = layers.BatchNormalization()(z)

z = layers.UpSampling2D((2, 2))(z)      
z = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(z)
z = layers.BatchNormalization()(z)

z = layers.UpSampling2D((2, 2))(z)  
z = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(z)
z = layers.BatchNormalization()(z)

z = layers.UpSampling2D((2, 2))(z)   
outputs = layers.Conv2D(1, (1, 1), activation="sigmoid", padding="same")(z)

model_tl = models.Model(inputs, outputs)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True)

lr_reduce = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=2,
    min_lr=1e-6)

base_model.trainable = False

model_tl.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy")

history = model_tl.fit(
    x224_train, y224_train,
    validation_data=(x224_val, y224_val),
    epochs=15,
    batch_size=8,
    callbacks=[early_stop, lr_reduce])

base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

model_tl.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy")

history_ft = model_tl.fit(
    x224_train, y224_train,
    validation_data=(x224_val, y224_val),
    epochs=10,
    batch_size=8,
    callbacks=[early_stop, lr_reduce])

In [ ]:
model_tl.summary()

In [ ]:
model_tl.save("resnet50_model.keras")

## 📊 Results Comparison

### Quantitative Results (Test Set)

| Model | Input Size | Loss Function | Best Threshold | Validation IoU | Test IoU (Approx.) | Notes |
|------|-----------|---------------|----------------|----------------|-------------------|-------|
| Baseline CNN | 168 × 168 | BCE + Dice | **0.10** | **≈ 0.50** | ≈ 0.49–0.50 | Produces sharper boundary predictions |
| ResNet50 (Transfer Learning) | 224 × 224 | BCE | **0.15 – 0.20** | ≈ 0.48 | ≈ 0.47–0.48 | Strong encoder, limited decoder capacity |

---

### Qualitative Comparison

| Aspect | Baseline CNN | ResNet50 |
|------|-------------|----------|
| Boundary continuity | Better | Slightly fragmented |
| Thin edge detection | Better | Weaker |
| Over-smoothing | Low | Moderate |
| Visual similarity to GT | Higher | Moderate |

---

## 🧠 Project Summary

This project investigates **image boundary segmentation** on the **Berkeley Segmentation Dataset** using two deep learning approaches: a **custom baseline CNN** and a **ResNet50-based transfer learning model**.

The baseline CNN was designed as an encoder–decoder architecture trained from scratch and optimized using a **Binary Cross-Entropy (BCE) + Dice loss** to mitigate the severe class imbalance present in boundary segmentation tasks. The ResNet50 model employed a pretrained ImageNet encoder followed by a lightweight upsampling-based decoder.

Model performance was evaluated using the **Intersection over Union (IoU)** metric and qualitative visual comparisons of predicted boundary masks. Threshold tuning was applied to each model to determine the optimal binarization value.

### Key Findings
- The baseline CNN achieved the **highest performance**, reaching a validation IoU of approximately **0.50** at a threshold of **0.10**.
- The ResNet50-based model achieved a slightly lower validation IoU of approximately **0.48**, despite leveraging pretrained features.
- Qualitative evaluation showed that the baseline CNN produced **more continuous and accurate boundary predictions**, particularly for thin edges.
- The limited improvement of the ResNet model is attributed to the absence of **skip connections**, which are essential for preserving fine-grained spatial details in segmentation tasks.

### Conclusion
The results indicate that for **edge-based image segmentation**, a well-designed CNN trained from scratch with an appropriate loss function can outperform a deeper transfer learning model. Future work may explore **UNet-style architectures with skip connections** to further improve segmentation accuracy.
